Task 1: File Management Script

Write a Bash script that:
- Creates a directory named "backup" in the user's home directory^
- Copies all .txt files from the current directory into the "backup" directory^
- Appends the current date and time to the filenames of the copied files.




In [ ]:
#!/bin/bash

# Create a variable to store the current date and time
TIMESTAMP=$(date +"%Y-%m-%d_%H-%M-%S")

# Create the "backup" directory in the user's home directory if it doesn't already exist
mkdir -p ~/backup

# Loop through all .txt files in the current directory
for file in *.txt; do
    # Copy the file to the "backup" directory with the timestamp appended
    cp "$file" ~/backup/"${file%.txt}_$TIMESTAMP.txt"
done

Task 2: System Health Check

Create a script that:
- Checks the system’s CPU and memory usage^
- Reports if the CPU usage is above 80% or if the available memory is below 20%^
- Logs the results to a file named system_health.log.

In [ ]:
#!/bin/bash

# Get CPU and memory usage
CPU_USAGE=$(top -bn1 | grep "Cpu(s)" | sed "s/.*, *\([0-9.]*\)%* id.*/\1/" | awk '{print 100 - $1}')
FREE_MEMORY=$(free -m | awk 'NR==2{printf "%.2f", $NF*100/$2 }') # Percentage of free memory

LOG_FILE="system_health.log"
ALERT=""

# Check CPU usage
if (( $(echo "$CPU_USAGE > 80" | bc -l) )); then
    ALERT+="High CPU usage: ${CPU_USAGE}%\n"
fi

# Check available memory
if (( $(echo "$FREE_MEMORY < 20" | bc -l) )); then
    ALERT+="Low free memory: ${FREE_MEMORY}%\n"
fi

# Log results
echo "$(date): System Health Check" >> "$LOG_FILE"
echo "CPU Usage: ${CPU_USAGE}%" >> "$LOG_FILE"
echo "Free Memory: ${FREE_MEMORY}%" >> "$LOG_FILE"

if [ -n "$ALERT" ]; then
    echo "Alerts:" >> "$LOG_FILE"
    echo -e "$ALERT" >> "$LOG_FILE"
    echo "System health issues detected. Check $LOG_FILE for details."
else
    echo "System health is good."
fi
echo "" >> "$LOG_FILE" # Add a newline for readability

Task 3: User Account Management

Write a script that:
- Reads a list of usernames from a file (e.g., user_list.txt)^
- Creates a new user for each username^
- Generates a random password for each user and saves the username and password to a file named
credentials.txt.

In [ ]:
# Create a dummy user_list.txt for testing
nano user_list.txt
user1
user2
user3

echo "Created dummy user_list.txt"


In [ ]:
#!/bin/bash

USER_LIST_FILE="user_list.txt"
CREDENTIALS_FILE="credentials.txt"

# Clear credentials file or create if it doesn't exist
> "$CREDENTIALS_FILE"

# Check if the user list file exists
if [ ! -f "$USER_LIST_FILE" ]; then
    echo "Error: User list file '$USER_LIST_FILE' not found."
    exit 1
fi

# Read usernames from the file
while IFS= read -r username; do
    # Skip empty lines
    if [ -z "$username" ]; then
        continue
    fi

    # Generate a random password
    password=$(openssl rand -base64 12)

    # Create the user (if they don't exist)
    if id "$username" &>/dev/null; then
        echo "User '$username' already exists. Skipping creation."
    else
        useradd "$username"
        echo "Created user: $username"
    fi

    # Set the user's password
    echo "$username:$password" | chpasswd

    # Save credentials to the file
    echo "Username: $username, Password: $password" >> "$CREDENTIALS_FILE"

done < "$USER_LIST_FILE"

echo "User account management complete. Credentials saved to $CREDENTIALS_FILE"

Task 4: Automated Backup

Create a script that:
- Takes a directory path as input from the user^
- Compresses the directory into a .tar.gz file^
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

In [ ]:
#!/bin/bash

# Prompt the user for the directory path
read -p "Enter the path of the directory to back up: " BACKUP_DIR

# Validate input - Check if the directory exists
if [ ! -d "$BACKUP_DIR" ]; then
    echo "Error: Directory '$BACKUP_DIR' not found."
    exit 1
fi

# Generate archive name with current date
DATE=$(date +"%Y-%m-%d")
ARCHIVE_NAME="backup_${DATE}.tar.gz"

# Compress the directory
tar -czf "$ARCHIVE_NAME" -C "$(dirname "$BACKUP_DIR")" "$(basename "$BACKUP_DIR")"

# Confirm backup
if [ $? -eq 0 ]; then
    echo "Backup of '$BACKUP_DIR' created successfully as '$ARCHIVE_NAME'"
else
    echo "Error: Backup failed."
fi

Task 5: Simple To-Do List

Create a Bash script that:
- Implements a simple command-line to-do list^
- Allows the user to add tasks, view tasks, and remove tasks^
- Saves the tasks to a file (e.g., todo.txt).

In [ ]:
#!/bin/bash

TODO_FILE="todo.txt"

# Initialize todo file if it doesn't exist
if [ ! -f "$TODO_FILE" ]; then
    touch "$TODO_FILE"
    echo "Created empty todo list: $TODO_FILE"
fi

# Function to add a task
add_task() {
    read -p "Enter the task: " task
    echo "$task" >> "$TODO_FILE"
    echo "Task added."
}

# Function to view tasks
view_tasks() {
    if [ ! -s "$TODO_FILE" ]; then
        echo "Your to-do list is empty."
    else
        echo "To-Do List:"
        nl -s ". " "$TODO_FILE"
    fi
}

# Function to remove a task
remove_task() {
    view_tasks
    if [ -s "$TODO_FILE" ]; then
        read -p "Enter the number of the task to remove: " task_number
        # Check if the input is a number and within the range of tasks
        if [[ "$task_number" =~ ^[0-9]+$ ]] && [ "$task_number" -ge 1 ] && [ "$task_number" -le $(wc -l < "$TODO_FILE") ]; then
            # Use sed to delete the line based on the task number
            sed -i "${task_number}d" "$TODO_FILE"
            echo "Task $task_number removed."
        else
            echo "Invalid task number."
        fi
    fi
}

# Main menu loop
while true; do
    echo ""
    echo "Simple To-Do List"
    echo "1. Add Task"
    echo "2. View Tasks"
    echo "3. Remove Task"
    echo "4. Exit"
    read -p "Enter your choice: " choice

    case $choice in
        1)
            add_task
            ;;
        2)
            view_tasks
            ;;
        3)
            remove_task
            ;;
        4)
            echo "Exiting."
            exit 0
            ;;
        *)
            echo "Invalid choice. Please enter a number between 1 and 4."
            ;;
    esac
done

Task 6: Automated Software Installation

Write a script that:
- Reads a list of software package names from a file (e.g., packages.txt)^
- Installs each package using the appropriate package manager (apt, yum, etc.)^
- Logs the installation status of each package.

In [ ]:
# Create a dummy packages.txt for testing
nano packages.txt
nano
htop
tree
echo "Created dummy packages.txt"

In [ ]:
#!/bin/bash

PACKAGES_FILE="packages.txt"
LOG_FILE="installation_status.log"

# Clear log file or create if it doesn't exist
> "$LOG_FILE"

# Detect package manager
if command -v apt-get &> /dev/null; then
    PKG_MANAGER="apt-get"
    INSTALL_CMD="install -y"
elif command -v yum &> /dev/null; then
    PKG_MANAGER="yum"
    INSTALL_CMD="install -y"
elif command -v dnf &> /dev/null; then
    PKG_MANAGER="dnf"
    INSTALL_CMD="install -y"
else
    echo "Error: No supported package manager found (apt, yum, dnf)." | tee -a "$LOG_FILE"
    exit 1
fi

echo "Detected package manager: $PKG_MANAGER" | tee -a "$LOG_FILE"
echo "Reading packages from $PACKAGES_FILE" | tee -a "$LOG_FILE"
echo "---" | tee -a "$LOG_FILE"

# Check if the package list file exists
if [ ! -f "$PACKAGES_FILE" ]; then
    echo "Error: Package list file '$PACKAGES_FILE' not found." | tee -a "$LOG_FILE"
    exit 1
fi

# Read packages from the file and install
while IFS= read -r package; do
    # Skip empty lines and comments
    if [ -z "$package" ] || [[ "$package" =~ ^# ]]; then
        continue
    fi

    echo "Attempting to install: $package" | tee -a "$LOG_FILE"
    # Use the detected package manager to install the package
    if sudo "$PKG_MANAGER" "$INSTALL_CMD" "$package"; then
        echo "Status: Success" | tee -a "$LOG_FILE"
    else
        echo "Status: Failed" | tee -a "$LOG_FILE"
    fi
    echo "---" | tee -a "$LOG_FILE"

done < "$PACKAGES_FILE"

echo "Automated software installation complete. Status logged to $LOG_FILE"

Task 7: Text File Processing

Create a script that:
- Takes a text file as input^
- Counts and displays the number of lines, words, and characters in the file^
- Finds and displays the longest word in the file.

In [ ]:
#!/bin/bash

# Prompt the user for the file path
read -p "Enter the path of the text file to process: " TEXT_FILE

# Validate input - Check if the file exists
if [ ! -f "$TEXT_FILE" ]; then
    echo "Error: File '$TEXT_FILE' not found."
    exit 1
fi

# Count lines, words, and characters
NUM_LINES=$(wc -l < "$TEXT_FILE")
NUM_WORDS=$(wc -w < "$TEXT_FILE")
NUM_CHARS=$(wc -m < "$TEXT_FILE") # Use -m for character count (handles multi-byte characters)

# Find the longest word
# This command sequence:
# 1. Converts all whitespace to newlines
# 2. Removes empty lines
# 3. Sorts by length in reverse order
# 4. Takes the first line (which is the longest word)
# 5. Removes leading/trailing whitespace
LONGEST_WORD=$(tr -s '[:space:]' '\n' < "$TEXT_FILE" | sed '/^$/d' | awk '{ print length, $0 }' | sort -rn | head -n 1 | cut -d ' ' -f 2-)

# Display results
echo "--- File Analysis ---"
echo "File: $TEXT_FILE"
echo "Number of lines: $NUM_LINES"
echo "Number of words: $NUM_WORDS"
echo "Number of characters: $NUM_CHARS"
echo "Longest word: \"$LONGEST_WORD\""
echo "---------------------"